<a href="https://colab.research.google.com/github/GeorgiaDimaki/COMMA/blob/master/agentic_shopping_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛒 Agentic Shopping Assistant

This notebook will go through all the steps to create an agentic shopping assistant. \
We will:
* Connect to OpenAI
* Create a simple agent
* Create an MCP server
* Create an MCP client
* Create another agent using an agentic framework (LangGraph)

Afterwards, we will look at a real-world application of agentic technologies: a coding assistant.

<br/>
<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/assistant.png" width="600">

# ⚙️ Setup
Before we start using OpenAI models, you need to set an API key. \
If you don't already have an key, you can generate one at: https://platform.openai.com/api-keys.

Save the key as a Colab Secret variable called "OPENAI_API_KEY":
1. Click on the key icon in the left bar menu.
2. Click on `+ Add new secret`.
3. Name the variable and paste the key in the value field.
4. Enable notebook access.

<br/>
<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/colab_setup.png" width="600">



Import the API key into the notebook.

In [1]:
import os

# Detect if running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Set API key based on environment
if IN_COLAB:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
else:
    # For local Jupyter: ensure OPENAI_API_KEY is set in your environment
    if "OPENAI_API_KEY" not in os.environ:
        print("Warning: OPENAI_API_KEY not found in environment variables")

Then, make a test call to OpenAI.

In [2]:
import openai
client = openai.OpenAI()
model = "gpt-4o"

# Test that the LLM is set up correctly
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Say 'OK' if you can read this."}],
    max_tokens=10
)
print(f"LLM test: {response.choices[0].message.content}")

LLM test: OK


# 🤖 Creating a Shopping Assistant agent

Start with an empty cart.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/cart_empty.png" width="100">

In [3]:
from collections import defaultdict

MY_SHOPPING_CART = defaultdict(lambda: 0)

Provide a tool to add items.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/cart_insert.png" width="100">

In [4]:
def add_to_cart(item_name: str, quantity: int=1) -> str:
  """Tool: "Add an grocery item to the shopping cart."."""
  try:
    MY_SHOPPING_CART[item_name] += quantity
    return f"Added {quantity} {item_name}."
  except Exception as ex:
    return f"Failed to insert '{quantity}'. {ex!r}"


# Test: add an item to the cart
add_to_cart("apples", 5)
MY_SHOPPING_CART

defaultdict(<function __main__.<lambda>()>, {'apples': 5})

Provide a tool to remove items.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/cart_remove.png" width="100">

In [5]:
def remove_from_cart(item_name: str) -> str:
  """Tool: Remove an item from the cart."""
  try:
    if item_name in MY_SHOPPING_CART:
      del MY_SHOPPING_CART[item_name]
      return f"Removed {item_name}."
    else:
      return f"{item_name} is not in the cart."
  except Exception as ex:
    return f"Failed to remove '{item_name}'. {ex!r}"


# Test: remove an item from the cart
remove_from_cart("apples")
MY_SHOPPING_CART

defaultdict(<function __main__.<lambda>()>, {})

And finally, a tool to see what's in the cart.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/cart_view.png" width="120">

In [6]:
def whats_in_the_cart() -> str:
  """Tool: Get the contents of the cart."""
  try:
    empty = True
    for item, quantity in MY_SHOPPING_CART.items():
      if quantity > 0:
        empty = False
        break
    if empty:
      return "The cart is empty."
    else:
      return f"Here's the cart: {dict(sorted(MY_SHOPPING_CART.items()))}"
  except Exception as ex:
    return f"Failed to get cart contents. {ex!r}"


# Test: look inside the cart
whats_in_the_cart()

'The cart is empty.'

Now, imagine you are the agent, and you receive the following prompts.

In [7]:
SYSTEM_PROMPT = """
You are a helpful assistant that adds and removes items from a shopping cart.

You have access to tools that let you:
1. Add grocery items
2. Remove grocery items
3. Inspect what is in the cart

Please do not ask me any follow-up questions after I make a request to you.
Just use the tools to satisfy my request.
"""

user_prompt = "Please add all the ingredients for a fruit salad into my shopping cart."

What do you do?

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_0.png" width="800">

</br></br>
</br></br>
</br></br>
</br></br>

To get started, the LLM needs to know the prompt, and which tools are available.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_1.png" width="600">



We need a description for each tool.

In [8]:
tools: list[dict] = [
    {
        "type": "function",
        "function": {
            "name": "add_to_cart",
            "description": "Add a grocery item to the shopping cart.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item_name": {
                        "type": "string",
                        "description": "The grocery item to be added."
                    },
                    "quantity": {
                        "type": "integer",
                        "description": "The quantity of the grocery item to be added."
                    },
                },
                "required": ["item_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "remove_from_cart",
            "description": "Remove an item from the cart.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item_name": {
                        "type": "string",
                        "description": "The grocery item to be removed."
                    },
                },
                "required": ["item_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "whats_in_the_cart",
            "description": "Get the contents of the shopping cart.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
]

We also need to map each tool's name to the corresponding function.

In [9]:
from typing import Dict, Callable

tool_map: Dict[str, Callable] = {
    "add_to_cart": add_to_cart,
    "remove_from_cart": remove_from_cart,
    "whats_in_the_cart": whats_in_the_cart
}

If the LLM decides to run a tool, instead of responding with regular text, it will respond with a `tool_call` object.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_2.png" width="600">

In [10]:
from types import SimpleNamespace

# This tool_call object matches the format expected by OpenAI
tool_call = SimpleNamespace(
    id="call_abc123",
    function=SimpleNamespace(
        name="add_to_cart",
        arguments='{"item_name":"apple", "quantity": 5}'
    ),
    type="function"
)

Parse the tool name and the arguments.

In [11]:
import json

function_name = tool_call.function.name
print(f"{function_name=}")

arguments = json.loads(tool_call.function.arguments)
print(f"{arguments=}")

function_name='add_to_cart'
arguments={'item_name': 'apple', 'quantity': 5}


Important! Verify that the tool requested is one of the available tools.



In [12]:
# List all available tools
allowed_tool_names = [tool["function"]["name"] for tool in tools]

# Check if the tool is available
if function_name not in allowed_tool_names:
    print(f"Error: '{function_name}' is not an allowed tool.")
else:
    print(f"'{function_name}' is an allowed tool.")

'add_to_cart' is an allowed tool.


Call the function with the unpacked arguments and print its response.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_3.png" width="600">

In [13]:
tool_func = tool_map[function_name]
response = tool_func(**arguments)
response

'Added 5 apple.'

Combine all tool calling steps in one method.

In [14]:
import json

def execute(tool_call) -> str:
    """Execute a tool call and return the result, if any."""
    # Extract the function name from the tool call
    function_name = tool_call.function.name

    # Parse the arguments from JSON string to dictionary
    arguments = json.loads(tool_call.function.arguments)

    # Important! Verify that the function is one of the allowed tools
    allowed_tool_names = [tool["function"]["name"] for tool in tools]
    if function_name not in allowed_tool_names:
        return f"Error: '{function_name}' is not an allowed tool."

    # Call the function with the unpacked arguments
    tool_func = tool_map[function_name]
    response = tool_func(**arguments)

    # Return the tool's response
    return response

## The agentic loop
Once the tool call is complete, report its outcome to the LLM.

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_4.png" width="600">

The LLM might respond with more instructions! \
Then, the agent will operate in a `while True` loop, calling tools when the LLM requests them.  

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_5.png" width="600">


All interactions between the agent, the LLM and the tools should be recorded in a message log. \
We initialize it here.

In [15]:
messages = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": "Please add all the ingredients for lasagna into my shopping cart."
    }
]

The following utility function prints out the log.

In [16]:
import textwrap

def print_log(messages, width=70):
    """Pretty print messages log."""
    # For each message in the log
    for i, msg in enumerate(messages):
        print(f"\n[{i}] Role: {msg['role']}")

        # Display message text if present
        if msg.get('content'):
            for line in str(msg['content']).split('\n'):
                wrapped = textwrap.fill(line, width=width, initial_indent='    ', subsequent_indent='    ')
                print(wrapped)

        # Display tool calls if present
        if msg.get('tool_calls'):
            for tc in msg['tool_calls']:
                func_name = tc.function.name
                func_args = tc.function.arguments
                print(f"    🔧 {func_name}({func_args})")


print_log(messages)


[0] Role: system

    You are a helpful assistant that adds and removes items from a
    shopping cart.

    You have access to tools that let you:
    1. Add grocery items
    2. Remove grocery items
    3. Inspect what is in the cart

    Please do not ask me any follow-up questions after I make a
    request to you.
    Just use the tools to satisfy my request.


[1] Role: user
    Please add all the ingredients for lasagna into my shopping cart.


Ask the agent what to do next.

In [17]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
).choices[0].message

# Display the raw tool call in the agent's response
tool_calls_data = [tc.model_dump() for tc in response.tool_calls]
print(json.dumps(tool_calls_data, indent=2))

[
  {
    "id": "call_k7sVrgxQTBVDm8RlAomhStdj",
    "function": {
      "arguments": "{\"item_name\": \"lasagna noodles\", \"quantity\": 1}",
      "name": "add_to_cart"
    },
    "type": "function"
  },
  {
    "id": "call_uTTBd9eaTMGXWraHB82gZ8np",
    "function": {
      "arguments": "{\"item_name\": \"ground beef\", \"quantity\": 1}",
      "name": "add_to_cart"
    },
    "type": "function"
  },
  {
    "id": "call_a7ZgMUd42aepAP3Cq1HDob00",
    "function": {
      "arguments": "{\"item_name\": \"ricotta cheese\", \"quantity\": 1}",
      "name": "add_to_cart"
    },
    "type": "function"
  },
  {
    "id": "call_9EaeB5KSqjG1guZ5KnPEzgSp",
    "function": {
      "arguments": "{\"item_name\": \"mozzarella cheese\", \"quantity\": 1}",
      "name": "add_to_cart"
    },
    "type": "function"
  },
  {
    "id": "call_v58Zx0PvSNRBGDDUuWwcO3BU",
    "function": {
      "arguments": "{\"item_name\": \"Parmesan cheese\", \"quantity\": 1}",
      "name": "add_to_cart"
    },
    "type

Update the chat history with the agent's response.

In [18]:
messages.append({
    "role": "assistant",
    "content": response.content,
    "tool_calls": response.tool_calls
})

print_log(messages)


[0] Role: system

    You are a helpful assistant that adds and removes items from a
    shopping cart.

    You have access to tools that let you:
    1. Add grocery items
    2. Remove grocery items
    3. Inspect what is in the cart

    Please do not ask me any follow-up questions after I make a
    request to you.
    Just use the tools to satisfy my request.


[1] Role: user
    Please add all the ingredients for lasagna into my shopping cart.

[2] Role: assistant
    🔧 add_to_cart({"item_name": "lasagna noodles", "quantity": 1})
    🔧 add_to_cart({"item_name": "ground beef", "quantity": 1})
    🔧 add_to_cart({"item_name": "ricotta cheese", "quantity": 1})
    🔧 add_to_cart({"item_name": "mozzarella cheese", "quantity": 1})
    🔧 add_to_cart({"item_name": "Parmesan cheese", "quantity": 1})
    🔧 add_to_cart({"item_name": "tomato sauce", "quantity": 1})
    🔧 add_to_cart({"item_name": "onion", "quantity": 1})
    🔧 add_to_cart({"item_name": "garlic", "quantity": 1})
    🔧 add_to_

Execute the first tool call.

In [19]:
MY_SHOPPING_CART

outcome = execute(response.tool_calls[0])
outcome

'Added 1 lasagna noodles.'

Append the outcome to the message history.

In [20]:
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(outcome)
})
print_log(messages)


[0] Role: system

    You are a helpful assistant that adds and removes items from a
    shopping cart.

    You have access to tools that let you:
    1. Add grocery items
    2. Remove grocery items
    3. Inspect what is in the cart

    Please do not ask me any follow-up questions after I make a
    request to you.
    Just use the tools to satisfy my request.


[1] Role: user
    Please add all the ingredients for lasagna into my shopping cart.

[2] Role: assistant
    🔧 add_to_cart({"item_name": "lasagna noodles", "quantity": 1})
    🔧 add_to_cart({"item_name": "ground beef", "quantity": 1})
    🔧 add_to_cart({"item_name": "ricotta cheese", "quantity": 1})
    🔧 add_to_cart({"item_name": "mozzarella cheese", "quantity": 1})
    🔧 add_to_cart({"item_name": "Parmesan cheese", "quantity": 1})
    🔧 add_to_cart({"item_name": "tomato sauce", "quantity": 1})
    🔧 add_to_cart({"item_name": "onion", "quantity": 1})
    🔧 add_to_cart({"item_name": "garlic", "quantity": 1})
    🔧 add_to_

Combine all steps into one method.

In [21]:
def submit_request(
    user_prompt: str,
    verbose: bool = True
    ):
    """Submit a request to the agent and run any tools it calls."""
    # Initialize the chat history
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    while True:

        # Ask the agent what to do next
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        ).choices[0].message

        # Update the chat history with the agent's response
        messages.append({
            "role": "assistant",
            "content": response.content,
            "tool_calls": response.tool_calls
        })

        # If agent did not call any tools, we are done
        if not response.tool_calls:
            if verbose:
              print(f"\n⭐ {whats_in_the_cart()}")
            break

        # Execute all tool calls
        for tool_call in response.tool_calls:
            if verbose:
              print(f"\n🔧 The agent is calling a tool: "
                  f"{tool_call.function.name}"
                  f"({json.loads(tool_call.function.arguments)})")

            # Append the outcome to the message history
            outcome = execute(tool_call)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(outcome)
            })

## Let's test the agent!

Start with an empty cart.

In [22]:
MY_SHOPPING_CART = defaultdict(lambda: 0)

Ask the agent to add some items.

In [23]:
submit_request("Please add all the ingredients for lasagna into the cart.")


🔧 The agent is calling a tool: add_to_cart({'item_name': 'Lasagna noodles', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Tomato sauce', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Ground beef', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Ricotta cheese', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Mozzarella cheese', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Parmesan cheese', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Onion', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Garlic', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Olive oil', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Salt', 'quantity': 1})

🔧 The agent is calling a tool: add_to_cart({'item_name': 'Black Pepper', 'quantity': 1})

🔧 The agent is callin

Ask the agent to remove some items.

In [24]:
submit_request("Please remove pepper from the cart.")


🔧 The agent is calling a tool: remove_from_cart({'item_name': 'pepper'})

⭐ Here's the cart: {'Basil': 1, 'Black Pepper': 1, 'Garlic': 1, 'Ground beef': 1, 'Lasagna noodles': 1, 'Mozzarella cheese': 1, 'Olive oil': 1, 'Onion': 1, 'Parmesan cheese': 1, 'Ricotta cheese': 1, 'Salt': 1, 'Tomato sauce': 1}


This "manual" way of managing an agent is called *direct orchestration*. \
It's helpful to understand how an agent works under the hood! \
Later in this notebook, we will create a similar agent with less code, using an open-source framework.

# 🗄️ Creating an MCP Server

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/calling_tools.png" width="700">

</br>


</br>
</br>
</br>

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/mcp_tool_calling.png" width="700">

Create the MCP server.

In [27]:
%pip install --quiet mcp
from mcp.server import Server
from mcp.types import Tool, TextContent

server = Server("set-server")
print("✓ Server created")

✓ Server created


Create an MCP wrapper for listing the available tools.

In [28]:
async def list_tools() -> list[Tool]:
    """Return the list of available tools from our tools definition."""
    # Create an empty list to store MCP Tool objects
    mcp_tools = [
      Tool(
        name="add_to_cart",
        description="Add a grocery item to the shopping cart.",
        inputSchema={
            "type": "object",
            "properties": {
                "item_name": {
                    "type": "string",
                    "description": "The grocery item to be added."
                },
                "quantity": {
                    "type": "integer",
                    "description": "The quantity of the grocery item to be added."
                },
            },
            "required": ["item_name"]
        }
      ),

      Tool(
          name="remove_from_cart",
          description="Remove an item from the cart.",
          inputSchema={
              "type": "object",
              "properties": {
                  "item_name": {
                      "type": "string",
                      "description": "The grocery item to be removed."
                  },
              },
              "required": ["item_name"]
          }
      ),

      Tool(
          name="whats_in_the_cart",
          description="Get the contents of the shopping cart.",
          inputSchema={
              "type": "object",
              "properties": {},
              "required": []
          }
      )
    ]

    return mcp_tools

# Register the list_tools function with the server
server.list_tools()(list_tools)

<function __main__.list_tools() -> list[mcp.types.Tool]>

Create an MCP wrapper for executing tools.

In [29]:
from typing import Dict, Callable

tool_map: Dict[str, Callable] = {
    "add_to_cart": add_to_cart,
    "remove_from_cart": remove_from_cart,
    "whats_in_the_cart": whats_in_the_cart
}


MY_SHOPPING_CART = defaultdict(lambda: 0)
MY_SHOPPING_CART

In [30]:
from types import SimpleNamespace

async def call_tool(name: str, arguments: dict) -> list[TextContent]:
  """Handle MCP tool calls by delegating to our existing tools."""

  # Important! Verify that the function is one of the allowed tools
  allowed_tool_names = tool_map.keys()

  if name not in allowed_tool_names:
      return f"Error: '{name}' is not an allowed tool."

  tool_func = tool_map[name]
  result = tool_func(**arguments)

  # Convert result to MCP response format
  return [TextContent(type="text", text=str(result))]

# Register the call_tool function with the server
server.call_tool()(call_tool)

<function __main__.call_tool(name: str, arguments: dict) -> list[mcp.types.TextContent]>

Create a web application.

In [31]:
# FastAPI is a framework for building REST APIs
%pip install --quiet fastapi
from mcp.server.sse import SseServerTransport
from fastapi import FastAPI, Request
from fastapi.responses import Response

# Create a FastAPI web application
app = FastAPI()

Expose a `POST` endpoint, used to handle incoming tool calls.

In [32]:
# Create an SSE transport that will handle messages at the "/messages" path
sse = SseServerTransport("/messages")

# Mount the POST handler for receiving messages
# Clients send messages to http://host:port/messages
app.mount("/messages", sse.handle_post_message)

Expose a `GET` endpoint, used to establish the connection to the server.

In [33]:
async def handle_sse(request: Request):
    """Handle incoming SSE connections from MCP clients."""
    # Connect the SSE transport to get read/write streams
    async with sse.connect_sse(
        request.scope, request.receive, request._send
    ) as (read_stream, write_stream):
        # Run the MCP server with these streams
        await server.run(
            read_stream,
            write_stream,
            server.create_initialization_options()
        )
    return Response()

# Register the GET endpoint with the FastAPI app
# Clients connect to http://host:port/sse to establish SSE connection
app.add_api_route("/sse", handle_sse, methods=["GET"])

### Set up to start and stop the server

In [34]:
# Uvicorn is a web server that handles HTTP requests and asynchronous code
%pip install --quiet uvicorn psutil

import uvicorn, time, os, psutil
from multiprocessing import Process


def get_pid_on_port(port):
    '''Utility method to get the process ID running at a given port.'''
    for conn in psutil.net_connections(kind='inet'):
        if conn.laddr.port == port and conn.status == 'LISTEN':
            if conn.pid != os.getpid():
                return conn.pid
    return None


def start_server(app_obj, port):
    '''Idempotent function to start the server at a given port.'''
    pid = get_pid_on_port(port)
    if pid:
        print(f"✗ Server already running at http://127.0.0.1:{port}")
        return

    p = Process(target=uvicorn.run, args=(app_obj,),
                kwargs={'host': "0.0.0.0", 'port': port, 'log_level': "warning"},
                daemon=True)
    p.start()
    time.sleep(3)
    if p.is_alive():
        print(f"✓ Server started at http://127.0.0.1:{port}")
    else:
        print("✗ Server failed to start.")


def stop_server(port):
    '''Idempotent function to stop the server at a given port.'''
    pid = get_pid_on_port(port)
    if not pid:
        print(f"✗ No server running at http://127.0.0.1:{port}.")
        return
    try:
        proc = psutil.Process(pid)
        proc.terminate()
        proc.wait(timeout=3)
        print("✓ Server stopped.")
    except psutil.TimeoutExpired:
        print("✗ Server is taking too long to exit.")



### Start the web server.

In [35]:
server_port = 12345
stop_server(server_port) # Make sure that no server is running
start_server(app, server_port)

server_url = f"http://127.0.0.1:{server_port}/sse"

✗ No server running at http://127.0.0.1:12345.
✓ Server started at http://127.0.0.1:12345


# 🤝 Creating an MCP Client

List available tools on the MCP server.

In [35]:
from mcp import ClientSession
from mcp.client.sse import sse_client

# Utility function to list tools
async def list_mcp_tools():
    """Helper function to list available MCP tools."""
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            available_tools = await session.list_tools()
            return available_tools

available_tools = await list_mcp_tools()
print("Available tools:", [t.name for t in available_tools.tools])

Available tools: ['add_to_cart', 'remove_from_cart', 'whats_in_the_cart']


Test each tool, starting with an empty set.

In [36]:
# Utility function to call a tool
async def call_mcp_tool(tool_name: str, arguments: dict = {}):
    """Helper function to call an MCP tool."""
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            result = await session.call_tool(tool_name, arguments)
            return result

Add "cherries" to the cart.

In [37]:
result = await call_mcp_tool("add_to_cart", {"item_name": "cherries"})
print(result.content[0].text)

Added 1 cherries.


Attempt to remove "bananas" from the set.

In [38]:
result = await call_mcp_tool("remove_from_cart", {"item_name": "bananas"})
print(result.content[0].text)

bananas is not in the cart.


Read all contents of the set.

In [39]:
result = await call_mcp_tool("whats_in_the_cart")
print(result.content[0].text)

Here's the cart: {'cherries': 1}


In [40]:

MY_SHOPPING_CART = defaultdict(lambda: 0)
MY_SHOPPING_CART

defaultdict(<function __main__.<lambda>()>, {})

In [41]:
result = await call_mcp_tool("whats_in_the_cart")
print(result.content[0].text)

Here's the cart: {'cherries': 1}


# 🧠 Orchestration with LangGraph

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/langchain_ecosystem.png" width="700">

^: *[From Models to Pipelines: A Practical Guide to Data Modeling, Architectures, and Engineering Tools](https://aws.plainenglish.io/from-models-to-pipelines-a-practical-guide-to-data-modeling-architectures-and-engineering-tools-49a3ff58bc87)*

In [ ]:
%pip install --quiet --upgrade "langchain" "langchain-openai" "langgraph" "langchain-mcp-adapters"

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Create MCP client that connects to your set-server
client = MultiServerMCPClient(
    {
        "set-server": {
            "transport": "sse",
            "url": f"http://localhost:{server_port}/sse",
        }
    }
)

Get available tools from the MCP server.

In [ ]:
tools_from_mcp = await client.get_tools()

for tool in tools_from_mcp:
    print(f"- {tool.name}: {tool.description}")

Create a LangGraph agent.

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

agent_executor = create_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    tools_from_mcp,
)

The following function calls the LangGraph agent and prints the conversation history.

In [ ]:
async def submit_langgraph_request(user_prompt: str, verbose: bool = True):
    """Submit a request to the LangGraph agent and display the conversation."""
    from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

    # Run the agent with the system prompt and user's prompt
    result = await agent_executor.ainvoke({
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    })

    # Display the conversation if verbose
    if verbose:
        for message in result['messages']:

            # Print user message
            if isinstance(message, HumanMessage):
                print("\n👤 User: " + message.content)

            # Print agent message
            elif isinstance(message, AIMessage):
                if message.content:
                    print("\n🤖 Agent: " + message.content)

            # Print tool action
            elif isinstance(message, ToolMessage):
                # Extract text from content (handle both string and list of dicts)
                if isinstance(message.content, str):
                    outcome_text = message.content
                elif isinstance(message.content, list) and len(message.content) > 0:
                    # Extract 'text' field from the first item if it's a dict
                    outcome_text = message.content[0].get('text', str(message.content[0]))
                else:
                    outcome_text = str(message.content)

                print(f"\n🔧 {message.name}: {outcome_text}")

Let's test it!

In [ ]:
await submit_langgraph_request("I would rather shop for a greek dinner."
                               "Can you fill up my shopping cart with the "
                               "ingredients for Moussaka and take out all that "
                               "Italian stuff that I don't need.")

In [ ]:
await submit_langgraph_request(
    "I'm American. How about just a hot dog with mustard."
    "Can you take out all those other ingredients too")

# 👩🏻‍💻 Creating a Coding Assistant

Here's what the outdated application looks like:
https://dwoodlock.github.io/Metric-Treadmill-2017/

## Get the Code

In [ ]:
CODE_DIRECTORY = "./Metric-Treadmill-2017"

In [ ]:
# Clone the Metric-Treadmill-2017 repo
from pathlib import Path
import shutil
import subprocess

path = Path("./Metric-Treadmill-2017")
if path.exists() and path.is_dir(): shutil.rmtree(path)

subprocess.run(["git", "clone", "--quiet", "https://github.com/dwoodlock/Metric-Treadmill-2017.git"],
               check=True)

## Set up the Prompts

In [ ]:
system_prompt = """
  You are a helpful assistant that can interact with a computer using tools.

  CRITICAL REQUIREMENT: You MUST ALWAYS provide your reasoning BEFORE
  calling any tool.

  For EVERY action you take:
  1. First, write a THOUGHT section in your text response explaining what
  you plan to do and why
  2. Then, after your THOUGHT, make the tool call

  NEVER call a tool without first explaining your reasoning in text.
  The content field of your response must NEVER be empty.

  Example:
  THOUGHT: I need to see what files exist in the directory to understand the
   codebase structure.
  [then call bash_tool with ls command]
  """

In [ ]:
# get the user prompt template
import httpx
try:
    response = httpx.get(
        "https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/"
        "refs/heads/main/prompts/coding-user-prompt-template.txt")
    response.raise_for_status()
    user_prompt_template = response.text
except httpx.HTTPError as e:
    print(f"Error: {e}")

print("\n".join(user_prompt_template.split("\n")[0:10]), "\n...") # print first 10 lines

## Updated the Tools - Add the Bash Tool

In [ ]:
import platform

def bash_tool(command: str) -> str:
    """Execute a bash command and return the result."""
    if not command.isascii():
        return "Error: Command contains non-ASCII characters."
    try:
        result = subprocess.run(
            command, shell=True, check=True,
            capture_output=True, text=False,  # Get bytes first
            cwd=CODE_DIRECTORY
        )

        # Try to decode as UTF-8
        try:
            output = result.stdout.decode('utf-8')
        except UnicodeDecodeError:
            # Binary data detected - provide helpful error
            return (
                "Error: Command output contains binary data (non-text file). "
                "Try filtering to text files only, e.g.:\n"
                "- Add file type filters: -name '*.css' -o -name '*.js'\n"
                "- Or use: file <filename> | grep -q text && cat <filename>"
            )

        return output if output else (
            "Command executed successfully with no output."
        )
    except subprocess.CalledProcessError as e:
        # Decode stderr, handling potential binary data there too
        try:
            stderr = e.stderr.decode('utf-8') if e.stderr else str(e)
        except UnicodeDecodeError:
            stderr = "Error output contains binary data"
        return f"Error executing command: {stderr}"

bash_description = f"""
Execute bash commands on the system.

## Useful command examples

### Create a new file:
cat <<'EOF' > newfile.py
import numpy as np
hello = "world"
print(hello)
EOF

### Edit files with sed:
{'IMPORTANT: You are on MacOS. Use `sed -i \'\'` instead of `sed -i`.' if platform.uname().system == 'Darwin' else ''}

# Replace all occurrences
sed -i {'\'\'' if platform.uname().system == 'Darwin' else ''} 's/old_string/new_string/g' filename.py

# Replace only first occurrence
sed -i {'\'\'' if platform.uname().system == 'Darwin' else ''} 's/old_string/new_string/' filename.py

# Replace first occurrence on line 1
sed -i {'\'\'' if platform.uname().system == 'Darwin' else ''} '1s/old_string/new_string/' filename.py

# Replace all occurrences in lines 1-10
sed -i {'\'\'' if platform.uname().system == 'Darwin' else ''} '1,10s/old_string/new_string/g' filename.py

### View file content:
# View specific lines with numbers
nl -ba filename.py | sed -n '10,20p'

### Any other command you want to run
You can run any bash command including ls, cat, find, python, etc.
""".strip()

In [ ]:
# Add the Bash tool to our tools list.  (Check if it's been added already in case the cell is re-run)

if 'bash_tool' not in [tool['function']['name'] for tool in tools]:
    tools.append(
{
    'type': 'function',
    'function': {
        'name': 'bash_tool',
        'description': bash_description,
        'parameters': {
            'type': 'object',
            'properties': {
                'command': {
                    'type': 'string',
                    'description': 'The bash command to execute.'
                }
            },
            'required': ['command']
        }
    }
})
tool_map['bash_tool'] = bash_tool

In [ ]:
# need to stop the server and restart it so it sees the new tools
stop_server(server_port)
start_server(app, server_port)

[tool.name for tool in await client.get_tools()]

## Get the User Task

In [ ]:
user_prompt = """
I have a web app that I wrote many years ago that converts my running plans
into a metric system if I find myself on a treadmill internationally.
I’m concerned that it uses old libraries and old approaches and it’ll just
stop working one day.  Can you modernize this app for me.  I'd especially
like you to eliminate unneeded and deprecated libraries and use modern
language features and approaches.
""".strip()

In [ ]:
# Insert the user request into the user prompt template to create the full prompt
enhanced_user_prompt = user_prompt_template.replace(
    "{{user-prompt}}",
    user_prompt
).replace(
    "{{platform-uname}}",
    str(platform.uname()))

## Set up a new LangGraph agent

In [ ]:
tools_from_mcp = await client.get_tools()

In [ ]:
# Create a LangGraph agent using the updated tools and higher-end model
from langchain.agents import create_agent

my_coding_agent = create_agent(
    ChatOpenAI(model="gpt-5.2", temperature=0),
    tools_from_mcp,
)

print("✓ LangGraph coding agent created")

In [ ]:
# get initial message ready (the agent already knows about the tools from above)

initial_message = {
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": enhanced_user_prompt}]}

In [ ]:
# here's the function to step through the events and print out each one as it's happening

async def run_coding_agent():

    iteration = 0

    async for event in my_coding_agent.astream(initial_message):
        iteration = iteration + 1
        print("-" * 40, "\nIteration", iteration)
        print_event(event)

    return event

In [ ]:
# A helper function to print out the details of the event

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
import textwrap

def print_event(event):
    # Set a reasonable wrap width
    wrap_width = 120  # Adjust this to your preference

    if 'model' in event:
        messages = event['model']['messages']
    elif 'tools' in event:
        messages = event['tools']['messages']
    else:
        assert False, "Unimplemented event structure"

    for message in messages:
        if isinstance(message, AIMessage):
            if type(message.content) == str:
                wrapped = textwrap.fill(message.content, width=wrap_width,
                                       initial_indent="AI: \033[34m",
                                       subsequent_indent="    ")
                print(wrapped + "\033[0m")
            elif type(message.content) == list:
                for content_block in message.content:
                    if content_block['type'] == 'tool_use':
                        call_str = f"AI: call {content_block['name']} {content_block['input']}"
                        wrapped = textwrap.fill(call_str, width=wrap_width, subsequent_indent="    ")
                        print("\033[34m" + wrapped + "\033[0m")
                    elif content_block['type'] == 'text':
                        wrapped = textwrap.fill(content_block['text'], width=wrap_width,
                                               initial_indent="AI: \033[34m ",
                                               subsequent_indent="    ")
                        print(wrapped + "\033[0m")
            else:
                assert False, "Unimplemented message.content type"
        elif isinstance(message, ToolMessage):
            if type(message.content) == list:
                for content_block in message.content:
                    if content_block['type'] == 'text':
                        wrapped = textwrap.fill(content_block['text'], width=wrap_width,
                                               initial_indent="Tool Result:\n\033[32m",
                                               subsequent_indent="")
                        print(wrapped + "\033[0m")
                    else:
                        assert False, "Unimplemented message.content type"
            else:
                assert False, "Unimplemented message.content type"
        else:
            assert False, "Unimplemented message type"

        if hasattr(message, 'tool_calls'):
            for tool_call in message.tool_calls:
                call_str = f"AI: TOOL CALL REQUEST: {tool_call['name']} {tool_call['args']}"
                wrapped = textwrap.fill(call_str, width=wrap_width, subsequent_indent="    ")
                print(wrapped)

In [ ]:
# before we run the whole loop, let's take a look at just one iteration

event = await anext(my_coding_agent.astream(initial_message))
event

In [ ]:
# slightly prettier form
print_event(event)

In [ ]:
print("Here we go!")

In [ ]:
result = await run_coding_agent()

## Let's take a look at the new site!

In [ ]:
import os
import time
from IPython.display import display, HTML
from random import randint
import subprocess
from google.colab.output import eval_js

query_param = f"?v={randint(10000, 99999)}"

# Kill ANY process using port 8000
!fuser -k 8000/tcp 2>/dev/null || true

# Wait a moment for port to be released
time.sleep(1)

# Start server in background
proc = subprocess.Popen(
    ['python', '-m', 'http.server', '8000', '--directory', 'Metric-Treadmill-2017'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(2)

# Check if process is still running
if proc.poll() is None:
    print("Server started successfully!")
    # Get the proxy URL
    base_url = eval_js("google.colab.kernel.proxyPort(8000)")
    full_url = f"{base_url}{query_param}"
    display(HTML(f'<a href="{full_url}" target="_blank" style="font-size: 18px;">Click here to view your website</a>'))
else:
    print("Server failed to start!")
    stdout, stderr = proc.communicate()
    print("STDOUT:", stdout.decode())
    print("STDERR:", stderr.decode())

In [ ]:
# download the resulting website so you can take a look.
if IN_COLAB:
  !zip -q -r -9 metric-treadmill.zip ./Metric-Treadmill-2017 \
    -x "**/.git/**" "**/node_modules/**" "**/.DS_Store"

  from google.colab import files
  files.download("metric-treadmill.zip")

# 🧹 MCP Cleanup

Stop the MCP server.

In [ ]:
# Stop the MCP server using the ServerManager
stop_server(server_port)

# Thank you!

###